# SETUP 

In [ ]:
import sys
import os
from pathlib import Path

modulepath = Path.cwd().parent / "utils"
modulepath = str(modulepath)
projectpath = Path.cwd().parent
projectpath = str(projectpath)
if modulepath not in sys.path:
    sys.path.insert(0,str(modulepath))
if projectpath not in sys.path:
    sys.path.insert(0,str(projectpath))  
print(sys.path)

# TRAIN

In [ ]:
import torch
from torch import nn

from utils.engine import train
from utils.model_architectures import TinyVGG_1
from utils.data_loaders import create_train_test_dataloaders
from utils.helpers import create_writer
from utils.helpers import create_dataframe
from utils.helpers import hyperparameter_combinations
from utils.helpers import save_dataframe
from utils.helpers import create_and_save_experiment_metadata

device = "cuda" if torch.cuda.is_available() else "cpu"
#next(model.parameters()).is_cuda #check if model on cuda

#from torchinfo import summary
#summary(tvgg,input_size=[32,1,28,28])

# A Hyperparameter named Iter controls the number of iterations for a given
# set of hyperparameters
HYPERPARAMETERS = {'Hidden_Channels': [30],
                   'Epochs': [10],
                   'lr': [0.1,0.01,0.001],
                   'Iter': [3],
                   'Size':[1]
                   }

experiment_name = "lr_sweep_size_01"
hp_combinations = hyperparameter_combinations(HYPERPARAMETERS)
    hp_keys = HYPERPARAMETERS.keys()

for n,combination in enumerate(hp_combinations):
    HIDDEN_CHANNELS = combination[0]
    NUM_EPOCHS = combination[1]
    LR = combination[2]
    ITER = combination[3]
    SIZE = combination[4]

    train_dataloader, test_dataloader = create_train_test_dataloaders(size=SIZE) #type: ignore

    model_0 = TinyVGG_1(input_shape=1,hidden_channels=HIDDEN_CHANNELS,output_shape=10).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(params=model_0.parameters(),lr=LR)

    model_name = model_0._get_name()
    #model_name = "Model_3"

    extra = ""
    namelist = [str(key)+"_"+str(val)+"_" for key,val in zip(hp_keys,combination)]
    for elem in namelist:
        extra += elem
    extra = extra[:-1]

    # Create a writer to save results to tensorboard
    writer = create_writer(experiment_name=experiment_name,
                           model_name=model_name,
                           extra=extra)
    # Train the model for a number of epochs and log the results to tensorboard
    results = train(model=model_0,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    epochs=NUM_EPOCHS,
                    iters=ITER,
                    writer=writer)
    # Store additional information on the experiment
    create_and_save_experiment_metadata(experiment_name=experiment_name,
                                        model_name=model_name,
                                        extra=extra,
                                        train_dataloader=train_dataloader,
                                        test_dataloader=test_dataloader,
                                        model=model_0,
                                        optimizer=optimizer,
                                        loss_fn=loss_fn,
                                        epochs=NUM_EPOCHS,
                                        hyperparameters_tuple=combination,
                                        hyperparameters_keys=hp_keys,
                                        )
    # Store the results of the accuracy and loss as a dataframe
    df = create_dataframe(results=results,
                          hyperparameters_tuple=combination,
                          hyperparameters_keys=hp_keys)
    save_dataframe(df=df,
                   model_name=model_name,
                   experiment_name=experiment_name,
                   extra=extra)


# PLOT 

In [ ]:
%matplotlib ipympl
from utils.helpers import retrieve_results
from utils.helpers import plot_results

experiment_name = None #"Test_plots"
model_name = None #"TinyVGG_1"
extra = None #"Hidden_Channels_40_Epochs_6_lr_0.01"

#Add functionality to input more than one experimemt name

#Add a toggle for experiment and model names
#No toggle for extra since they are too messy
#But maybe a filter of some kind


resdf = retrieve_results(experiment_name=experiment_name,model_name=model_name,extra=extra)
plot_results(results=resdf,experiment_name=experiment_name,model_name=model_name) # type: ignore